The file `1_SCC.txt` contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the 11th row looks like: "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646.

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Output Format: You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes).  (Note also that your answer should not have any spaces in it.)

WARNING: Because of the size of the graph you may have to manage memory carefully. 

In [ ]:
# Kosaraju's Two-Pass Algorithm: 
# Given directed graph G
# Let G_rev = G with all arcs reversed
# run DFS-Loop on G_rev to compute "magical" ordering of nodes
# Let f(v) = "finishing time" of each vertex v
# run DFS-Loop on G processing nodes in decreasing order of finishing times to discover the SCCs one-by-one

# DFS-Loop (graph G)
# global variable t = 0 [# of nodes processed so far] - for finishing times in 1st pass
# global variable s = NULL [current source vertex] - for leaders in 2nd pass
# Assume nodes labelled 1 to n
# for i=n...1:
#     if i not yet explored
#         s := i
#         DFS(G,i)

# DFS (graph G, node i)
# mark i explored (for rest of DFS-Loop)
# set leader(i) := node s
# for each (i,j) in G:
#     if j not yet explored:
#         DFS(G,j)
# t++
# set f(i) := t (ith finishing time)

In [1]:
# while writing the code, I encountered the maximum recursion depth exceeded error 
# max recursion error protects us against a stack overflow. This is when the pointer in a stack exceeds 
# the stack bound. Without this error, our program would try to use more memory space than was available.
# print recursion depth limit
import sys
sys.getrecursionlimit()

3000

In [ ]:
# you can override the default recursion limit Python sets
# you should be careful when you use this method because it may cause a stack overflow 
# depending on the resources available to the Python interpreter.
#DFS_Loop(tails = heads, heads = tails, node_labels = list(range(874931, 874930, -1)), N = N)
# got to len of nodes_explored:  6,656 with default recursive length of 3000
# got to len of nodes_explored: 27,770 with resursive length 12000
# got to len of nodes_explored: 51,184 with resursive length 20000
# got to len of nodes_explored: 142,751 with resursive length 40000 but kernel died
# for one node i = 874931, 615,722 is the length of nodes explored from that node 
# and the iterative algorithm took more than a day
sys.setrecursionlimit(40000)

In [ ]:
sys.getrecursionlimit()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def DFS_recursive(tails, heads, i):
    """
    Depth-first search recursive function.
    
    Input parameters:
    tails -- list of tail nodes in the input directed graph
    heads -- list of head nodes in the input directed graph
    i -- vertex i from which to start DFS
    
    Output:
    f(i) -- ith node finishing time
    
    """
    # global declaration is required is we are changing t value inside the function
    global t
    # mark i explored (for rest of DFS-Loop)
    nodes_explored.append(i)

    # set leader(i) := node s
    # leaders is indexed 0 to N-1
    # so for node N, assign its leader to N-1 position in the numpy array
    leaders[i-1] = s
    
    # indices where tail node is i
    ii = np.where(tails == i)[0]

    # tail nodes from i to j
    jj = heads[ii]
    
    # for each (i,j) in G:
    for arc in jj:
        # if j not yet explored:
        if arc not in nodes_explored:
            # DFS(G,j)
            DFS(tails, heads, arc)
            
    # t++
    t += 1
    # set f(i) := t (ith finishing time)
    f[i-1] = t

In [ ]:
# another way to deal with recursion depth limit is
# to rewrite the function as a iterative 

In [ ]:
def DFS_iterative(tails, heads, i):
    """
    Depth-first search function replacing recursion with a loop
    to avoid stack overflow errors and maximum recusion depth limits
    set by Python because of memory limitation. Choose this function
    when dealing with a large graph.
    
    Input parameters:
    tails -- list of tail nodes in the input directed graph
    heads -- list of head nodes in the input directed graph
    i -- vertex i from which to start DFS
    
    Output:
    f(i) -- ith node finishing time
    
    """
        
    # global declaration is required as we are changing t value inside the function
    global t
 
    current_nodes = [i]
    
    # collect all nodes that DFS would recurse on to assign f values
    nodes_to_assign = []
    
    while bool(current_nodes) :
        # pop last item from list
        i = current_nodes.pop()
        
        if i not in nodes_explored:
            nodes_explored.append(i)
            print('number of nodes explored: ', len(nodes_explored))

            nodes_to_assign.append(i) 
            
            leaders[i-1] = s
            ii = np.where(tails == i)[0]
            if ii.size != 0:
                jj = heads[ii]
                for arc in jj:
                    current_nodes.append(arc)


    for n in reversed(nodes_to_assign):
        t += 1
        f[n-1] = t
        

In [ ]:
def DFS_Loop(tails, heads, node_labels, N):
    """
    Depth-first search loop.
    
    Input parameters:
    tails -- list of tail nodes in the input directed graph
    heads -- list of head nodes in the input directed graph
    node_labels -- order of nodes to process
    N -- number of nodes in the input graph
    """
    global s
   
    # global variable t = 0 [# of nodes processed so far] - for finishing times in 1st pass
    # global variable s = NULL [current source vertex] - for leaders in 2nd pass
    # Assume nodes labelled 1 to n
    # for i=n...1:
    for i in node_labels:
        print('i in DFS_Loop', i)
        # if i not yet explored
        if i not in nodes_explored:
            # s := i
            s = i
            # DFS(G,i)
            #DFS_recursive(tails, heads, i)
            DFS_iterative(tails, heads, i)


In [ ]:
def Kosarajus(input_file, N):
    """
    Kosaraju's Two-Pass Algorithm.
    
    Input parameters:
    input_file -- a text file containing two columns for tails and heads of the vertices
    N -- number of nodes in the given graph
    
    Output:
    dictionary sorted by decreasing order of SCC's in a given graph
    """
    global nodes_explored, leaders, t, f
    
    # number of vertices
    N = N
    
    # read in input graph
    with open(input_file) as f:
        lines = f.readlines()
    
    # these two arrays represend the input directed graph G
    tails = np.empty(len(lines), dtype=np.int)
    heads = np.empty(len(lines), dtype=np.int)
    
    for i, line in enumerate(lines):
        tails[i] = line.split(' ')[0]
        heads[i] = line.split(' ')[1]
        
    nodes_explored = []
    
    # ith node leader
    leaders = np.empty(N, dtype=np.int)

    # ith finishing time 
    f = np.zeros(N, dtype=np.int)
    
    # number of nodes processed so far
    t = 0
    # current source vertex
    s = np.NaN
    
    # first pass of DFS_Loop
    # Let G_rev = G with all arcs reversed
    # run DFS-Loop on G_rev to compute "magical" ordering of nodes
    DFS_Loop(tails = heads, heads = tails, node_labels = list(range(N, 0, -1)), N = N)
    
    # reset 
    nodes_explored = []
    # ith node leader
    leaders = np.empty(N, dtype=np.int)
    # number of nodes processed so far
    t = 0
    # current source vertex
    s = np.NaN
    
    # relabel the nodes based on finishing time f
    tails_second_pass = np.empty(len(tails), dtype=np.int) 
    heads_second_pass = np.empty(len(heads), dtype=np.int) 
    
    for ff in f:
        # iterate over the finishing times 
        ii = np.where(tails == ff)[0]
        tails_second_pass[ii] = f[ff-1] 
        ii = np.where(heads == ff)[0]
        heads_second_pass[ii] = f[ff-1]
       
    # second pass of DFS_Loop
    # Let f(v) = "finishing time" of each vertex v
    # run DFS-Loop on G processing nodes in decreasing order of finishing times to discover the SCCs one-by-one
    DFS_Loop(tails = tails_second_pass, heads = heads_second_pass, node_labels=list(range(N, 0, -1)), N = N)
    
    # output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes
    scc = pd.DataFrame(leaders, columns = ["x"]).groupby('x').size().to_dict()
    result = {k: v for k, v in sorted(scc.items(), key=lambda item: item[1], reverse=True)}
    return result

In [ ]:
# for '1_SCC.txt' graph, the 5 largest SCC are:
# 615986: 434821,
# 617403: 968,
# 798411: 459,
# 161539: 313,
# 709991: 211

In [ ]:
%%time
# N = 875714
result = Kosarajus('1_SCC.txt', N = 875714)